In [1]:
!pwd
import sys
sys.path.append('/workspace')


/workspace/notebooks


In [2]:
from src.core.db.config import DatabaseEnum
from src.core.db.models.pdf_models import Fincen8300Rev4
from src.core.db.models.main_models import EmployeeToDocument
from src.core.db.session import DBContext, DbQuery

[21/03/2021 20:58:51] [    INFO] [dotenv.main] [_get_stream():77] [PID:99 TID:140612221933376] -> Python-dotenv could not find configuration file /workspace/src/core/env/.local.env.local.


# Read the pdf data into a data source

In [3]:
db = DbQuery(DatabaseEnum.PDF_INGESTION_DB)
result = db.execute("SELECT * from public.fincen8300_rev4")


[21/03/2021 20:58:51] [    INFO] [sqlalchemy.engine.base.Engine] [_execute_context():1235] [PID:99 TID:140612221933376] -> select version()
[21/03/2021 20:58:51] [    INFO] [sqlalchemy.engine.base.Engine] [_execute_context():1240] [PID:99 TID:140612221933376] -> {}
[21/03/2021 20:58:51] [    INFO] [sqlalchemy.engine.base.Engine] [_execute_context():1235] [PID:99 TID:140612221933376] -> select current_schema()
[21/03/2021 20:58:51] [    INFO] [sqlalchemy.engine.base.Engine] [_execute_context():1240] [PID:99 TID:140612221933376] -> {}
[21/03/2021 20:58:51] [    INFO] [sqlalchemy.engine.base.Engine] [_cursor_execute():1341] [PID:99 TID:140612221933376] -> SELECT CAST('test plain returns' AS VARCHAR(60)) AS anon_1
[21/03/2021 20:58:51] [    INFO] [sqlalchemy.engine.base.Engine] [_cursor_execute():1342] [PID:99 TID:140612221933376] -> {}
[21/03/2021 20:58:51] [    INFO] [sqlalchemy.engine.base.Engine] [_cursor_execute():1341] [PID:99 TID:140612221933376] -> SELECT CAST('test unicode returns

In [4]:
type(result)

sqlalchemy.engine.result.ResultProxy

In [5]:
from pandas import DataFrame
df = DataFrame(result.fetchall())
df.columns = result.keys()

In [6]:
df.head()

,id,amends_prior_report,suspicious_transaction,multiple_individuals,last_name,first_name,middle_initial,tin,address,dob,...,address_brc,city_brc,state_brc,zip_code_brc,nature_of_business_brc,signature_footer,title_footer,date_of_signature_footer,contact_name_printed_brc,contact_phone_brc
0,d08188b0-61f9-4e82-bb44-d2ca9e3f547d,None,None,None,Baranov,Jacqueline,None,761870877,500 east 77th street,01291971,...,None,None,None,None,None,None,None,None,None,None
1,fcc83438-4a34-47ad-adc8-52d7be146434,None,None,None,Burton,Cliffton,None,192837465,54 Longfellow St Apt 1986,02101962,...,None,None,None,None,None,None,None,02112021,Rachel Adams,None
2,186b1728-d9e9-4a89-8286-1a66b0d56ab0,None,None,None,Abowd,Daniel,None,171363880,10-30 Jackson Ave.,05301970,...,None,None,None,None,None,None,None,None,None,None
3,5f49c0a1-4d9e-468c-a2b1-6abe0b704ed4,None,None,None,Hetfield,James,None,111111115,415 North Grand Avenue,08031963,...,None,None,None,None,None,None,None,07111999,Venante Alcida,None
4,17513205-cff6-44c4-b7bf-447fb0a38d65,None,None,None,Hammett,Kirk,None,222222223,20 S. Green Hill Ave.,11181962,...,None,None,None,None,None,None,None,11222019,Ivan Alonzo,None


In [7]:
df.columns

Index(['id', 'amends_prior_report', 'suspicious_transaction',
       'multiple_individuals', 'last_name', 'first_name', 'middle_initial',
       'tin', 'address', 'dob', 'city', 'state', 'zip', 'country',
       'occ_prof_bus', 'identdoc_describe', 'identdoc_issued_by',
       'identdoc_number', 'multiple_behalf', 'last_name_org_name_behalf',
       'first_name_behalf', 'middle_initial_behalf', 'tin_behalf',
       'dba_behalf', 'address_behalf', 'occ_prof_bus_behalf', 'city_behalf',
       'state_behalf', 'zip_behalf', 'country_behalf',
       'identdoc_describe_behalf', 'identdoc_issued_by_behalf',
       'identdoc_number_behalf', 'date_cash_received', 'total_cash_received',
       'cash_received_mult_pmts', 'total_price', 'usde_us_currency',
       'usde_amt_benj_higher', 'usde_foreign_currency',
       'foreign_currency_country', 'usde_cashiers_checks', 'usde_money_orders',
       'usde_bank_drafts', 'usde_travelers_checks', 'type_ot_personal_prop',
       'type_ot_real_property', 

In [8]:
from src.sources.data_source import DataSource
fincen = DataSource(df)

# Read employee data into data source

In [9]:
db = DbQuery(DatabaseEnum.MAIN_INGESTION_DB)
result = db.execute("SELECT * from public.employee")

[21/03/2021 20:58:53] [    INFO] [sqlalchemy.engine.base.Engine] [_execute_context():1235] [PID:99 TID:140612221933376] -> select version()
[21/03/2021 20:58:53] [    INFO] [sqlalchemy.engine.base.Engine] [_execute_context():1240] [PID:99 TID:140612221933376] -> {}
[21/03/2021 20:58:53] [    INFO] [sqlalchemy.engine.base.Engine] [_execute_context():1235] [PID:99 TID:140612221933376] -> select current_schema()
[21/03/2021 20:58:53] [    INFO] [sqlalchemy.engine.base.Engine] [_execute_context():1240] [PID:99 TID:140612221933376] -> {}
[21/03/2021 20:58:53] [    INFO] [sqlalchemy.engine.base.Engine] [_cursor_execute():1341] [PID:99 TID:140612221933376] -> SELECT CAST('test plain returns' AS VARCHAR(60)) AS anon_1
[21/03/2021 20:58:53] [    INFO] [sqlalchemy.engine.base.Engine] [_cursor_execute():1342] [PID:99 TID:140612221933376] -> {}
[21/03/2021 20:58:53] [    INFO] [sqlalchemy.engine.base.Engine] [_cursor_execute():1341] [PID:99 TID:140612221933376] -> SELECT CAST('test unicode returns

In [10]:
print(result.keys())

['id', 'prefix_name', 'first_name', 'last_name', 'middle_name', 'suffix_name', 'date_of_birth', 'dod_id']


In [11]:
df_employee = DataFrame(result.fetchall())
df_employee.columns = result.keys()
df_employee.head()

,id,prefix_name,first_name,last_name,middle_name,suffix_name,date_of_birth,dod_id
0,c316a646-8955-489c-a685-6c2f5047a66d,None,Diane,Meier,None,Sr.,2057-02-28,7186
1,7dca02c2-d542-44ae-8b70-f63f3496f57e,Ms.,Samantha,Young,None,Sr.,1980-07-13,8955
2,6a47f014-f99b-4792-ac17-0ce1fb21965f,None,Adrian,Frandle,None,None,1982-12-20,5805
3,6a8926d7-8966-4ab8-b8ef-4bea2b6c334c,Ms.,Christine,Johnson,None,None,1991-04-07,6424
4,4600c711-f054-433a-b215-4ac5fbf72ccd,None,Kaisheng,Yao,None,Jr.,1972-02-17,8930


In [12]:
employee = DataSource(df_employee)

# Map the columns

In [13]:
# equivalent to _process_structured

from src.sources.structured_data_source import StructuredDataSource
from src.mapping.pdfs.pdf_field_name_classifier import FieldNameClassifier
from src.mapping.pdfs.pdf_field_label_catalog import FieldLabelCatalog
from src.mapping.pdfs.pseudofield_generator import PseudofieldGenerator

pseudofield_generator = PseudofieldGenerator(fincen)
pseudofield_generator.generate()


In [14]:
# equivalent to _create_column_relations_for
def _create_column_relations_for(source, target):
    """Create column relations from canonical column identifiers."""
    gold_id_info = FieldNameClassifier.get_id_info_from_df(target.get_data())
    data_id_info = FieldNameClassifier.get_id_info_from_df(source.get_data())
    for identifier in FieldLabelCatalog:
        if identifier in gold_id_info and identifier in data_id_info:
            g_id = gold_id_info[identifier]
            d_id = data_id_info[identifier]
            source.create_column_relation(
                d_id.field_name, g_id.field_name, target
            )
            print(
                "New relation detected: %s" % str(source.column_relations[-1])
            )

In [15]:
_create_column_relations_for(fincen, employee)

New relation detected: dob -> pandas DataFrame (hash 1661532706735119568): date_of_birth (1.00 conf)
New relation detected: first_name -> pandas DataFrame (hash 1661532706735119568): first_name (1.00 conf)
New relation detected: last_name -> pandas DataFrame (hash 1661532706735119568): last_name (1.00 conf)


# Find Row Mappings

In [16]:
from src.mapping.rows.row_mapping_configuration import RowMappingConfiguration
from src.mapping.values.value_matching_configuration import ValueMatchingConfiguration

In [17]:
import json
def load_config(path):
        """
        Read JSON from a filepath
        """
        with open(path, "r") as F:
            return json.load(F)

In [25]:
value_matching_config_json = load_config('../config/mapping/levenshtein_default.json')
row_mapping_config_json = load_config('../config/mapping/weighted_linear_default.json')
value_matching_config = ValueMatchingConfiguration(**value_matching_config_json)
row_mapping_config = RowMappingConfiguration(**row_mapping_config_json)

In [26]:
x = fincen.get_column_relations()
print(x)

[ColumnRelation(target_data_source=pandas DataFrame (hash 1661532706735119568), source_column_name='dob', target_column_name='date_of_birth', confidence=1.0), ColumnRelation(target_data_source=pandas DataFrame (hash 1661532706735119568), source_column_name='first_name', target_column_name='first_name', confidence=1.0), ColumnRelation(target_data_source=pandas DataFrame (hash 1661532706735119568), source_column_name='last_name', target_column_name='last_name', confidence=1.0)]


In [27]:
fincen.map_rows_to(
    employee, value_matching_config, row_mapping_config
)


In [21]:
import pandas as pd
def _generate_structured_row_matches(source, employee):
        """Generate structured row matches."""
        rows = {
            'first_name': [],  # just for sanity check
            'last_name': [],   # just for sanity check  
            'emp_uuid': [],
            'doc_uuid': []
        }
        target_df = employee.get_data()
        for relation in source.row_relations:
            source_index = relation.source_index
            target_index = relation.target_index
            source_row = source.get_data().iloc[source_index]
            target_row = employee.get_data().iloc[target_index]
            rows['emp_uuid'].append(target_row.id)
            rows["doc_uuid"].append(source_row.id)
            rows["first_name"].append(source_row.first_name)
            rows["last_name"].append(source_row.last_name)
        return pd.DataFrame(rows)

In [28]:
results_df = _generate_structured_row_matches(fincen, employee)
results_df.head()

,first_name,last_name,emp_uuid,doc_uuid
0,Cerena,Rodriquez,7cca73ad-6adb-4898-a6c7-7e56835709ae,69d0a22e-8293-45d8-80be-bed167dff23b
1,Cerena,Rodriquez,dcb6fe17-6374-4aa2-aba0-9e0cf6acabcd,69d0a22e-8293-45d8-80be-bed167dff23b
2,Cerena,Rodriquez,4d25f9e3-4767-4ac2-9cbe-594cad8d8a6b,69d0a22e-8293-45d8-80be-bed167dff23b
3,Cerena,Rodriquez,04307592-b5f4-4ebb-be72-e8d9ad6e9785,69d0a22e-8293-45d8-80be-bed167dff23b
4,Cerena,Rodriquez,92067627-73f3-4b0d-8a04-ec46619b1df4,69d0a22e-8293-45d8-80be-bed167dff23b


In [23]:
fincen.get_data().iloc[2].first_name


'Daniel'

# Write final dataframe to Main database

In [24]:
num_records = results_df.shape[0]
with DBContext(DatabaseEnum.MAIN_INGESTION_DB) as main_db:
    for i in range(num_records):
        row = results_df.iloc[i]
        main_db.add(EmployeeToDocument(employee_id=str(row.emp_uuid),
                                       document_ingestion_id=str(row.doc_uuid)
                                      ))

[21/03/2021 20:58:57] [    INFO] [sqlalchemy.engine.base.Engine] [_begin_impl():730] [PID:99 TID:140612221933376] -> BEGIN (implicit)
[21/03/2021 20:58:57] [    INFO] [sqlalchemy.engine.base.Engine] [_execute_context():1235] [PID:99 TID:140612221933376] -> INSERT INTO employee_to_document (employee_id, document_ingestion_id, related_employee_id) VALUES (%(employee_id)s, %(document_ingestion_id)s, %(related_employee_id)s) RETURNING employee_to_document.id
[21/03/2021 20:58:57] [    INFO] [sqlalchemy.engine.base.Engine] [_execute_context():1240] [PID:99 TID:140612221933376] -> {'employee_id': UUID('7cca73ad-6adb-4898-a6c7-7e56835709ae'), 'document_ingestion_id': UUID('69d0a22e-8293-45d8-80be-bed167dff23b'), 'related_employee_id': None}
[21/03/2021 20:58:57] [    INFO] [sqlalchemy.engine.base.Engine] [_execute_context():1235] [PID:99 TID:140612221933376] -> INSERT INTO employee_to_document (employee_id, document_ingestion_id, related_employee_id) VALUES (%(employee_id)s, %(document_ingest